# firstly the fasttext model

In [1]:
import fasttext
import pandas as pd
import csv
import random
from emoji import demojize
import re
from sklearn.metrics import classification_report

In [2]:
# preprocess hate data same way as abusive data and
# replace \n with space
with open('data/test_hate.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    # replace \n with space
    for i in range(len(data)):
        data[i][0] = re.sub(r'https.*[^ ]', 'URL', data[i][0])
        data[i][0] = re.sub(r'http.*[^ ]', 'URL', data[i][0])
        data[i][0] = re.sub(r'@([^ ]*)', '@USER', data[i][0])
        data[i][0] = demojize(data[i][0])
        data[i][0] = re.sub(r'(:.*?:)', r' \1 ', data[i][0])
        data[i][0] = re.sub(' +', ' ', data[i][0])
        data[i][0] = data[i][0].replace('\n', ' ')
    # write to csv
    with open('data/test_hate.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)



In [3]:
# divide test set in 303 hate tweets and 697 non-hate tweets
# Caution: Creates new different test set, for consistency use
# test_hate_sorted_definitive.csv

with open('data/test_hate.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    hate_data = []
    non_hate_data = []

    # first get al the hate tweets
    # yes this way is slower but it is easier to understand
    for row in data:
        if row[1] == 'y':
            hate_data.append(row)
    # now that we got the hate tweets, we can select at random the non-hate tweets
    for row in data:
        if row[1] == 'n':
            non_hate_data.append(row)

    # now create data set with all hate tweets and 697 non-hate tweets randomly selected
    # we will use this data set to test the model
    test_data = []
    for i in range(len(hate_data)):
        test_data.append(hate_data[i])

    # get random indexes
    random_indexes = random.sample(range(len(non_hate_data)), 697)
    # add the non-hate tweets to the test data
    for i in random_indexes:
        test_data.append(non_hate_data[i])
    # write to csv
    with open('data/test_hate_sorted.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(test_data)




In [4]:
def convert_to_fastext_format(path, delimiter):
    # convert csv to correct fasttext format
    with open(path, 'r') as f:
        reader = csv.reader(f, delimiter=delimiter)
        data = list(reader)
        # for every row but the first move the last column to beginning and add text '__label__'
        # and change both 'IMPLICIT' and 'EXPLICIT' to 'ABUSIVE'
        for i in range(1, len(data)):
            if data[i][1] == 'IMPLICIT':
                data[i][1] = 'ABUSIVE'
            elif data[i][1] == 'EXPLICIT':
                data[i][1] = 'ABUSIVE'

            # if prossesing the hate data
            if 'hate' in path:
                if data[i][1] == 'y':
                    data[i][1] = 'ABUSIVE'
                elif data[i][1] == 'n':
                    data[i][1] = 'NOT'
            data[i][0] = '__label__' + data[i][1] + ' ' + data[i][0]
            # remove the last column
            data[i] = data[i][:-1]

        if 'test' in path:
            new_path = path.replace('test', 'test_fasttext')
        if 'train' in path:
            new_path = path.replace('train', 'train_fasttext')
        if 'dev' in path:
            new_path = path.replace('dev', 'dev_fasttext')

        new_path = new_path.replace('.csv', '.txt')

        # write to file excluding the first row
        with open(new_path, 'w') as f:
            for row in data[1:]:
                f.write('\t'.join(row) + '\n')


In [5]:
convert_to_fastext_format('data/train_abusive.csv', '\t')
convert_to_fastext_format('data/test_abusive.csv', '\t')
convert_to_fastext_format('data/dev_abusive.csv', '\t')

In [6]:
convert_to_fastext_format('data/test_hate_sorted_definitive.csv', ',')

## train fasttext model

In [7]:
# tune fasttext parameters


In [2]:
# autotune the model

model = fasttext.train_supervised('data/train_fasttext_abusive.txt', autotuneValidationFile='data/dev_fasttext_abusive.txt', autotuneDuration=300, autotuneMetric="f1:__label__ABUSIVE")

Progress: 100.0% Trials:   90 Best score:  0.644444 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  23825
Number of labels: 2
words/sec/thread:  146423 lr:  0.046046 avg.loss:  0.396957 ETA:   0h 0m 6sProgress:  64.9% words/sec/thread:  148905 lr:  0.020642 avg.loss:  0.216720 ETA:   0h 0m 2s


In [3]:
# model = fasttext.train_supervised('data/train_fasttext_abusive.txt', epoch=25)
# save model
model.save_model('model/fasttext_model_300sec_FINAL.bin')

# load model
# model = fasttext.load_model('model/fasttext_model_bew.bin')

In [4]:
model.predict("@USER @USER @USER @USER @USER Dit is pure provocatie door die gehersenspoelde moslim.", k=-1)

(('__label__NOT', '__label__ABUSIVE'), array([0.94600821, 0.05401177]))

In [5]:
# test on test_abusive_fasttext.txt
model.test('data/test_fasttext_abusive.txt')

(1901, 0.7348763808521831, 0.7348763808521831)

In [6]:
# test on test_hate_fasttext.txt
model.test('data/test_fasttext_hate_sorted.txt')

(999, 0.7197197197197197, 0.7197197197197197)

In [7]:
from sklearn.metrics import confusion_matrix

with open ('data/test_hate_sorted_definitive.csv', 'r') as f:
    reader = csv.reader(f)
    data_hate = list(reader)
    # get predictions
    for i, row in enumerate(data_hate):
        row.append(model.predict(row[0], k=1)[0][0])
        if "NOT" in row[2]:
            data_hate[i][2] = "n"
        elif "ABUSIVE" in row[2]:
            data_hate[i][2] = "y"
    # write changes to file
    with open('data/test_hate_predictions.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data_hate)

    # and save to var
    predictions_hate = data_hate




In [8]:
# get predictions for test_abusive (and save in abusive predictions)

with open('data/test_abusive.csv', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    data_abuse = list(reader)
    # get predictions
    for i, row in enumerate(data_abuse[1:]):
        row.append(model.predict(row[0].replace('\n', ' '), k=1)[0][0])
        if "NOT" in row[2]:
            data_abuse[i+1][2] = "n"
        elif "ABUSIVE" in row[2]:
            data_abuse[i+1][2] = "y"
        if "NOT" in row[1]:
            data_abuse[i+1][1] = "n"
        elif "EXPLICIT" in row[1] or "IMPLICIT" in row[1]:
            data_abuse[i+1][1] = "y"
    # write changes to file
    with open('data/test_abusive_predictions.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data_abuse[1:])

    # and save to var
    predictions_abuse = data_abuse[1:]

In [9]:
y_true_hate = []
y_pred_hate = []
for row in predictions_hate:
    y_true_hate.append(row[1])
    y_pred_hate.append(row[2])


In [10]:
y_true_abuse = []
y_pred_abuse = []
for row in predictions_abuse:
    y_true_abuse.append(row[1])
    y_pred_abuse.append(row[2])

In [11]:
# create classification report for hate


print(classification_report(y_true_hate, y_pred_hate, digits=3))

              precision    recall  f1-score   support

           n      0.752     0.890     0.815       697
           y      0.562     0.327     0.413       303

    accuracy                          0.719      1000
   macro avg      0.657     0.608     0.614      1000
weighted avg      0.695     0.719     0.693      1000



In [12]:
# create classification report for abuse
print(classification_report(y_true_abuse, y_pred_abuse, digits=3))

              precision    recall  f1-score   support

           n      0.730     0.958     0.829      1264
           y      0.781     0.297     0.430       637

    accuracy                          0.736      1901
   macro avg      0.755     0.627     0.629      1901
weighted avg      0.747     0.736     0.695      1901



In [41]:
# show misclassified tweets hate
misclassified_hate = []
for row in predictions_hate:
    if row[1] != row[2]:
        misclassified_hate.append([row[0], row[1], row[2]])

import json
# write to json
json.dump(misclassified_hate, open('data/misclassified_hate.json', 'w'))

In [59]:
# count word 'moslim' in train data per class
# note, current output is not from 'moslim'
moslim_abuse = 0
moslim_not = 0
with open('data/train_abusive.csv', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    data_abuse = list(reader)
    for row in data_abuse[1:]:
        if "moslim" in row[0].lower():
            if "NOT" in row[1]:
                moslim_not += 1
            else:
                moslim_abuse += 1

print("moslim abuse:", moslim_abuse)
print("moslim not:", moslim_not)


moslim abuse: 15
moslim not: 7


# SVM model

In [2]:
from sklearn.svm import SVC

In [7]:
# load train data
with open('data/train_abusive.csv', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)
    text_only = []
    labels = []
    for row in data[1:]:
        text_only.append(row[0])
        if 'EXPLICIT' in row[1] or 'IMPLICIT' in row[1]:
            labels.append('ABUSIVE')
        else:
            labels.append('NOT')


In [8]:
# load abusive test data in same way above
with open('data/test_abusive.csv', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)
    test_text_only_abuse = []
    test_labels_abuse = []
    for row in data[1:]:
        test_text_only_abuse.append(row[0])
        if 'EXPLICIT' in row[1] or 'IMPLICIT' in row[1]:
            test_labels_abuse.append('ABUSIVE')
        else:
            test_labels_abuse.append('NOT')



In [10]:
# load hate test data in same way above
with open('data/test_hate_sorted_definitive.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    data = list(reader)
    test_text_only_hate = []
    test_labels_hate = []
    for row in data[1:]:
        test_text_only_hate.append(row[0])
        if 'y' in row[1]:
            test_labels_hate.append('ABUSIVE')
        else:
            test_labels_hate.append('NOT')

In [11]:
# first vecotrize the data with tf-idf word 2-grams and character 5-grams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

In [12]:
# create pipeline with 2-grams (Word) and 5-grams (Char)
pipe = Pipeline([
    ('union', FeatureUnion([
        ('word', TfidfVectorizer(ngram_range=(2, 2), analyzer='word')),
        ('char', TfidfVectorizer(ngram_range=(3, 5), analyzer='char')),
    ])),
    ('svm', SVC(kernel='linear', C=1.0)),
])


In [19]:
# do a grid search
from sklearn.model_selection import GridSearchCV

parameters = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'svm__kernel': ['linear'],
              }

# grid_search = GridSearchCV(pipe, param_grid=parameters) # best param is C=1
# grid_search.fit(text_only, labels)

#fit pipe to data (grid search resulted in C=1)
pipe.fit(text_only, labels)

Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('word',
                                                 TfidfVectorizer(ngram_range=(2,
                                                                              2))),
                                                ('char',
                                                 TfidfVectorizer(analyzer='char',
                                                                 ngram_range=(3,
                                                                              5)))])),
                ('svm', SVC(kernel='linear'))])

In [20]:
# get hate report
y_true_hate, y_pred_hate = test_labels_hate, pipe.predict(test_text_only_hate)
print(classification_report(y_true_hate, y_pred_hate, digits=3))

              precision    recall  f1-score   support

     ABUSIVE      0.595     0.291     0.391       302
         NOT      0.749     0.914     0.823       697

    accuracy                          0.726       999
   macro avg      0.672     0.603     0.607       999
weighted avg      0.702     0.726     0.692       999



In [21]:
# get abuse report
y_true_abuse, y_pred_abuse = test_labels_abuse, pipe.predict(test_text_only_abuse)
print(classification_report(y_true_abuse, y_pred_abuse, digits=3))

              precision    recall  f1-score   support

     ABUSIVE      0.848     0.272     0.411       637
         NOT      0.727     0.975     0.833      1264

    accuracy                          0.740      1901
   macro avg      0.787     0.624     0.622      1901
weighted avg      0.767     0.740     0.692      1901



## most frequent dummy classifier

In [5]:
from sklearn import dummy

dummy_clf_hate = dummy.DummyClassifier(strategy='most_frequent')
dummy_clf_abuse = dummy.DummyClassifier(strategy='most_frequent')
dummy_clf_abuse.fit(test_text_only_abuse, test_labels_abuse)
dummy_clf_hate.fit(test_text_only_hate, test_labels_hate)

DummyClassifier(strategy='most_frequent')

In [6]:
# get hate report
y_true_hate, y_pred_hate = test_labels_hate, dummy_clf_hate.predict(test_text_only_hate)
print(classification_report(y_true_hate, y_pred_hate, digits=3))

              precision    recall  f1-score   support

     ABUSIVE      0.000     0.000     0.000       302
         NOT      0.698     1.000     0.822       697

    accuracy                          0.698       999
   macro avg      0.349     0.500     0.411       999
weighted avg      0.487     0.698     0.573       999



/home/matisse/PycharmProjects/scriptie/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/matisse/PycharmProjects/scriptie/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/matisse/PycharmProjects/scriptie/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [8]:
# get abuse report
y_true_abuse, y_pred_abuse = test_labels_abuse, dummy_clf_abuse.predict(test_text_only_abuse)
print(classification_report(y_true_abuse, y_pred_abuse, digits=3))

              precision    recall  f1-score   support

     ABUSIVE      0.000     0.000     0.000       637
         NOT      0.665     1.000     0.799      1264

    accuracy                          0.665      1901
   macro avg      0.332     0.500     0.399      1901
weighted avg      0.442     0.665     0.531      1901



/home/matisse/PycharmProjects/scriptie/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/matisse/PycharmProjects/scriptie/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/matisse/PycharmProjects/scriptie/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo